In [7]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt

### 数据处理
#### ~~缺失值~~
#### 异常值
可以找几个变量画一下和房价的分布图，去掉异常的
#### ~~分类变量转变~~
### 建模
#### 变量筛选
没做，可以画图，选几个变量，或者用lasso、树模型之类的选变量
#### ~~xgb~~
训练集结果0.42
#### 加模型
去网上找或者在kaggle上公开代码里找

## 读取数据

In [8]:
train = pd.read_csv(r'data/train.csv')
test = pd.read_csv(r'data/test.csv')
# data = pd.concat([train, test], axis=0)
# data

## 数据处理

### 缺失值

In [9]:
def missing_value(data):
	## 缺失代表没有的
	data['Fence'] = data['Fence'].fillna('Na')
	data['Alley'] = data['Alley'].fillna('Na')
	data['MiscFeature'] = data['MiscFeature'].fillna('Na')
	data['PoolQC'] = data['PoolQC'].fillna('Na')
	data['FireplaceQu'] = data['FireplaceQu'].fillna('Na')
	data['BsmtHalfBath'] = data['BsmtHalfBath'].fillna(0)
	data['BsmtFullBath'] = data['BsmtFullBath'].fillna(0)
	## 缺失填众数的
	data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
	data['Functional'] = data['Functional'].fillna(data['Functional'].mode()[0])
	data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
	data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
	data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
	data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
	data['Utilities'] = data['Utilities'].fillna(data['Utilities'].mode()[0])
	'''
	BsmtFinSF1, BsmtFinType1, BsmtFinSF2, BsmtFinType2, BsmtQual
	都是地下室，面积就一个缺失值，面积是0的要么是未完工要么是缺失值，缺失值默认没有地下室
	BsmtCond地下室状态，面积为零的是没有，有面积的填众数TA
	BsmtExposure，面积为零的是没有，有面积的是没花园
	'''
	data['BsmtUnfSF'] = data['BsmtUnfSF'].fillna(0)
	data['BsmtFinSF1'] = data['BsmtFinSF1'].fillna(0)
	data['BsmtFinSF2'] = data['BsmtFinSF2'].fillna(0)
	data['BsmtFinType1'] = data['BsmtFinType1'].fillna('Na')
	data['BsmtFinType2'] = data['BsmtFinType2'].fillna('Na')
	data['TotalBsmtSF'] = data['BsmtUnfSF'] + data['BsmtFinSF1'] + data['BsmtFinSF2']
	data['BsmtQual'] = data['BsmtQual'].fillna('na')
	data.loc[(data['BsmtCond'].isnull()) & (data['TotalBsmtSF']==0), 'BsmtCond'] = 'Na'
	data.loc[(data['BsmtCond'].isnull()) & (data['TotalBsmtSF']!=0), 'BsmtCond'] = 'TA'
	data.loc[(data['BsmtExposure'].isnull()) & (data['TotalBsmtSF']==0), 'BsmtExposure'] = 'Na'
	data.loc[(data['BsmtExposure'].isnull()) & (data['TotalBsmtSF']!=0), 'BsmtExposure'] = 'No'
	'''
	LotFrontage：Linear feet of street connected to property
	居住在一起的人LotFrontage是相似的，我们将邻居的LotFrontage填充进来
	MSZoning: Identifies the general zoning classification of the sale
	住宅类型，应该也是和邻居一样，不过就缺了四个
	'''
	data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))
	data['MSZoning'] = data.groupby('Neighborhood')['MSZoning'].transform(lambda x:x.fillna(x.mode()[0]))
	'''
	GarageYrBlt：Year garage was built
	GarageArea：Size of garage in square feet
	GarageCars：Size of garage in car capacity
	分别表示车库的建造年份,面积,车位大小,这个值缺失很可能表示没有车库，用0填充；
	GarageYrBlt年份有个2207，应该是2007打错了，改回来
	车库的建造年份缺失值填最小值，年份越早越便宜，建的早比没有好（相关系数0.4）
	'''
	data.loc[data['GarageYrBlt']==2207, 'GarageYrBlt'] = 2007
	data['GarageYrBlt'] = data['GarageYrBlt'].fillna(data['GarageYrBlt'].min()-1)
	data['GarageArea'] = data['GarageArea'].fillna(0)
	data['GarageCars'] = data['GarageCars'].fillna(0)
	data['GarageType'] = data['GarageType'].fillna('Na')
	data.loc[(data['GarageCond'].isnull()) & (data['GarageArea']==0), 'GarageCond'] = 'Na'
	data.loc[(data['GarageCond'].isnull()) & (data['GarageArea']!=0), 'GarageCond'] = 'TA'
	data.loc[(data['GarageQual'].isnull()) & (data['GarageArea']==0), 'GarageQual'] = 'Na'
	data.loc[(data['GarageQual'].isnull()) & (data['GarageArea']!=0), 'GarageQual'] = 'TA'
	data.loc[(data['GarageFinish'].isnull()) & (data['GarageArea']==0), 'GarageFinish'] = 'Na'
	data.loc[(data['GarageFinish'].isnull()) & (data['GarageArea']!=0), 'GarageFinish'] = 'Unf'
	'''
	MasVnrArea,MasVnrType外墙类型和面积，面积缺失应该是没有，填0，对应类型无
	类型缺失有几个有面积的应该有东西，填众数
	'''
	data['MasVnrArea'] = data['MasVnrArea'].fillna(0)
	data.loc[(data['MasVnrType'].isnull()) & (data['MasVnrArea']==0), 'MasVnrType'] = 'None'
	data.loc[(data['MasVnrType'].isnull()) & (data['MasVnrArea']!=0), 'MasVnrType'] = 'BrkFace'
	return data

In [10]:
data = missing_value(train)

In [11]:
## 分类变量
cat_cols = list(data.select_dtypes(include = ['object']).columns) + ['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold']
## 年份变量
year_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
## 连续变量
ctn_cols = list(set(data.columns) - set(cat_cols) - set(year_cols) - set(['Id', 'SalePrice']))

## 年份变量取相对值
year_df_list = []
for col in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']:
    year_df = data[col] - 2010
    year_df_list.append(year_df)
year_data = pd.concat(year_df_list, axis=1)
## 分类变量取哑变量
cat_df_list = []
for col in cat_cols:
    df = pd.get_dummies(data[col], prefix=col)
    cat_df_list.append(df)
cat_data = pd.concat(cat_df_list, axis=1)\
## 拼接
data = pd.concat([year_data, cat_data, data[ctn_cols]], axis=1)
data

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,...,KitchenAbvGr,BsmtHalfBath,LotArea,BsmtFullBath,MiscVal,WoodDeckSF,TotRmsAbvGrd,GarageCars,HalfBath,FullBath
0,-7,-7,-7.0,-2,False,False,False,True,False,False,...,1,0,8450,1,0,0,8,2,1,2
1,-34,-34,-34.0,-3,False,False,False,True,False,False,...,1,1,9600,0,0,298,6,2,0,2
2,-9,-8,-9.0,-2,False,False,False,True,False,False,...,1,0,11250,1,0,0,6,2,1,2
3,-95,-40,-12.0,-4,False,False,False,True,False,False,...,1,0,9550,1,0,0,7,3,0,1
4,-10,-10,-10.0,-2,False,False,False,True,False,False,...,1,0,14260,1,0,192,9,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-11,-10,-11.0,-3,False,False,False,True,False,False,...,1,0,7917,0,0,0,7,2,1,2
1456,-32,-22,-32.0,0,False,False,False,True,False,False,...,1,0,13175,1,0,349,7,2,0,2
1457,-69,-4,-69.0,0,False,False,False,True,False,False,...,1,0,9042,0,2500,0,9,1,0,2
1458,-60,-14,-60.0,0,False,False,False,True,False,False,...,1,0,9717,1,0,366,5,1,0,1


## 建模

In [12]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(data,train['SalePrice'],train_size = 0.8)
#输出数据集大小
print ('原始数据集特征：',data.shape, 
       '\n训练数据集特征：',train_x.shape ,
      '\n测试数据集特征：',test_x.shape)

print ('\n原始数据集标签：',train['SalePrice'].shape, 
       '\n训练数据集标签：',train_y.shape ,
      '\n测试数据集标签：',test_y.shape)

原始数据集特征： (1460, 344) 
训练数据集特征： (1168, 344) 
测试数据集特征： (292, 344)

原始数据集标签： (1460,) 
训练数据集标签： (1168,) 
测试数据集标签： (292,)


In [33]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [34]:
model_xgb = xgb.XGBRegressor(n_estimators=2200)

In [35]:
n_folds = 5

def rmsle(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, data.values, train['SalePrice'], scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [36]:
model_xgb.fit(data, train['SalePrice'])
xgb_train_pred = model_xgb.predict(data)
xgb_pred = model_xgb.predict(test_x)
print(rmsle(train['SalePrice'], xgb_train_pred))

0.05591814918835381


In [38]:
model_xgb = xgb.XGBRegressor(reg_lambda=0.8571, n_estimators=2200, nthread = -1)

In [39]:
model_xgb.fit(data, train['SalePrice'])
xgb_train_pred = model_xgb.predict(data)
xgb_pred = model_xgb.predict(data)
print(rmsle(train['SalePrice'], xgb_train_pred))

0.042041779775907645


In [42]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso
import lightgbm as lgb

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [43]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, random_state =42)

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005,random_state=42))
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=42))

stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

In [44]:
model_xgb = xgb.XGBRegressor(n_estimators=2200, nthread = -1)
model_xgb.fit(data, train['SalePrice'])

model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=720)
model_lgb.fit(data, train['SalePrice'])

stacked_averaged_models.fit(data.values, train['SalePrice'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3516
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 235
[LightGBM] [Info] Start training from score 180921.195890


d:\conda\miniconda\envs\ml\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.581e+11, tolerance: 7.240e+08
  model = cd_fast.enet_coordinate_descent(
d:\conda\miniconda\envs\ml\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.741e+11, tolerance: 7.405e+08
  model = cd_fast.enet_coordinate_descent(
d:\conda\miniconda\envs\ml\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.838e+11, to

StackingAveragedModels(base_models=(Pipeline(steps=[('robustscaler',
                                                     RobustScaler()),
                                                    ('elasticnet',
                                                     ElasticNet(alpha=0.0005,
                                                                random_state=42))]),
                                    GradientBoostingRegressor(n_estimators=3000,
                                                              random_state=42),
                                    KernelRidge(alpha=0.6, coef0=2.5, degree=2,
                                                kernel='polynomial')),
                       meta_model=Pipeline(steps=[('robustscaler',
                                                   RobustScaler()),
                                                  ('lasso',
                                                   Lasso(alpha=0.0005,
                                                         random_state=42))]))

In [46]:
lgb_pred = model_lgb.predict(data)
xgb_pred = model_xgb.predict(data)
stacked_pred = stacked_averaged_models.predict(data.values)

In [47]:
print(rmsle(train['SalePrice'], lgb_pred))
print(rmsle(train['SalePrice'], xgb_pred))
print(rmsle(train['SalePrice'], stacked_pred))

1115.9903484195718
0.05591814918835381
10057.922013718942
